# Free Agent dataframes

En este apartado creamos la base de datos de los agente libres, pero agregada por equipo. Las secciones mostradas tienen los siguientes propósitos:
- Visualización de la data base.
- Creación de la base de datos de corte transversal con todos los datos para los agentes libres.

Importemos los modulos necesarios así como especificar la configuración deseada.

In [1]:
import pandas as pd
import numpy as np
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 5

In [3]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
os.chdir(path)

/home/usuario/Documentos/Github/Proyectos/MLB_HN/ETL


:Lo que se hará ahora es crear listas donde guardaremos los datos de interes de las bases de datos a lo largo del periodo de interes.

In [4]:
# Años de análisis
period = 11
csv = '.csv'
# Directorios
team_path = 'Data/New_Data/Teams/free_agents_team_'
pitcher_path = 'Data/New_Data/Pitchers/Free_Agent/free_agents_pitchers_'
hitter_path = 'Data/New_Data/Hitters/Free_Agent/free_agents_batters_'
# Originales:
df_team = [None]*period
df_pitcher = [None]*period
df_hitter = [None]*period
df_complete_pitcher = [None]*period
df_complete_hitter = [None]*period

Leeremos todos las bases de datos correspondientes

In [5]:
for i in range(0,period):
    df_team[i] = pd.read_csv(team_path + str(2011 + i) + csv)
    df_pitcher[i] = pd.read_csv(pitcher_path + str(2011 + i) + csv)
    df_hitter[i] = pd.read_csv(hitter_path + str(2011 + i) + csv)    

Veamos algunas bases de datos con las que trabajaremos

In [6]:
df_team[4].head()

,Equipo,Cantidad_agentes_libres,Valor_contrato,Acronimo,Victorias,Juegos totales,Promedio_victorias
0,Washington Nationals,8,216000000,WSH,83,161,0.515528
1,Chicago Cubs,13,190950000,CHC,96,161,0.596273
2,Boston Red Sox,8,186500000,BOS,78,161,0.484472
3,Chicago White Sox,13,137000000,CHW,76,161,0.472050
4,New York Yankees,7,101500000,NYY,87,161,0.540373


In [7]:
df_pitcher[7].head()

,Jugador,Anio,Status,Equipo_anterior,Anios_contrato,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Juegos_iniciados,...,Walks_2,Strike-outs_2,Wins_2,Losses_2,Saves_2,WHIP_2,ERA_2,Control_2,Comando_2,Juegos_2_2
0,Yu Darvish,2018,UFA,LAD,6,126000000,21000000,SP,8,8,...,441,2401,1,9,0,2.0164,24.5025,0.003403,5.444444,4096
1,Jake Arrieta,2018,UFA,CHC,3,75000000,25000000,SP,31,31,...,3249,19044,100,121,0,1.6641,15.6816,0.001345,5.861496,923521
2,Alex Cobb,2018,UFA,TB,4,57000000,14250000,SP,28,28,...,1849,10404,25,225,0,1.9881,24.0100,0.000984,5.626825,614656
3,Wade Davis,2018,UFA,CHC,3,52000000,17333333,RP,69,0,...,676,6084,9,36,1849,1.1236,17.0569,0.001957,9.000000,22667121
4,Tyler Chatwood,2018,UFA,COL,3,38000000,12666667,SP,25,20,...,9025,7225,16,36,0,3.2400,28.0900,0.010361,0.800554,390625


In [8]:
df_hitter[10].head()

,Jugador,Anio,Status,Equipo_anterior,Anios_contrato,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Porcetnaje_juegos,...,Sueldo,Sueldo_regular,Juegos_2,Porcetnaje_juegos_2,At-bats_2,Bateos_2,Home-runs_2,RBI_2,Porcentaje_bateo_2,OPS_2
0,George Springer,2021,UFA,HOU,6,150000000,25000000,CF,78,0.482,...,23666666,23666666,6084,0.232324,89401,6241,484,2500,0.069696,0.822649
1,J.T. Realmuto,2021,UFA,PHI,5,115500000,23100000,C,134,0.827,...,20000000,10000000,17956,0.683929,226576,15625,289,5329,0.069169,0.611524
2,Trevor Bauer,2021,UFA,CIN,3,102000000,34000000,SP,17,0.105,...,31333333,31333333,289,0.011025,900,1,0,1,0.001089,0.009604
3,D.J. LeMahieu,2021,UFA,NYY,6,90000000,15000000,3B,150,0.926,...,15000000,15000000,22500,0.857476,356409,25600,100,3249,0.071824,0.505521
4,Marcell Ozuna,2021,UFA,ATL,4,65000000,16250000,LF,48,0.298,...,16000000,16000000,2304,0.088804,35344,1600,49,676,0.045369,0.416025


Debido a que la columna en común son las que contienen los acrónimos de los equipos, se eliminará la columna con el nombre de los equipos en las bases de datos de los equipos con respecto a los agentes libres.

In [9]:
for i in range(0,period):
    df_team[i] = df_team[i][['Acronimo', 'Victorias', 'Juegos totales', 'Promedio_victorias']]
    names = ['Equipo', 'Victorias', 'Juegos totales', 'Promedio_victorias']
    df_team[i].columns = names

In [10]:
df_team[8].head()

,Equipo,Victorias,Juegos totales,Promedio_victorias
0,PHI,81,162,0.500000
1,SD,70,162,0.432099
2,WSH,93,162,0.574074
3,NYY,103,162,0.635802
4,LAD,105,161,0.652174


## Creación de las bases de datos con los datos completos

Se unir+an por medio de la columna de *Equipo*.

In [24]:
df_complete_pitcher[3] = pd.merge(df_pitcher[3], df_team[3], on = 'Equipo')

In [25]:
df_complete_pitcher[3]

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Juegos_iniciados,Inning_pitched,Bateos,Carreras,Carreras_ganadas,...,Wins,Losses,Saves,WHIP,ERA,Equipo,Sueldo,Victorias,Juegos totales,Promedio_victorias
0,Matt Garza,50000000,12500000,SP,27,27,163.3,143,77,66,...,8,8,0,1.18,3.64,MIL,12500000,82,162,0.506173
1,Francisco Rodriguez,3250000,3250000,RP,69,0,68.0,49,23,23,...,5,5,44,0.99,3.04,MIL,3250000,82,162,0.506173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,Pat Neshek,1000000,1000000,RP,71,0,67.3,44,14,14,...,7,2,6,0.79,1.87,STL,1000000,89,161,0.552795
67,Heath Bell,0,0,RP,13,0,17.3,24,16,14,...,1,1,0,1.85,7.27,TB,10000000,77,162,0.475309


In [26]:
for i in range(0,period):
    df_complete_pitcher[i] = pd.merge(df_pitcher[i], df_team[i], on = 'Equipo')
    df_complete_hitter[i] = pd.merge(df_hitter[i], df_team[i], on = 'Equipo')
    
    # Exportemos los dataframes por separado
    df_complete_hitter[i].to_csv('Data/New_Data/Free_Agent/Hitters/complete_free_agents_hitters_' + str(2011 + i) + '.csv', index = False)
    df_complete_pitcher[i].to_csv('Data/New_Data/Free_Agent/Pitchers/complete_free_agents_pitchers_' + str(2011 + i) + '.csv', index = False)

Veamos algunos de los productos finales

In [32]:
df_complete_hitter[9].head(10)

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Porcetnaje_juegos,At-bats,Bateos,Home-runs,RBI,Porcentaje_bateo,OPS,Equipo,Sueldo,Victorias,Juegos totales,Promedio_victorias
0,Josh Donaldson,92000000,23000000,3B,28,0.467,81,18,6,11,0.222,0.842,MIN,21000000,36,60,0.600000
1,Alex Avila,4250000,4250000,C,23,0.383,49,9,1,2,0.184,0.641,MIN,4250000,36,60,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,Marcell Ozuna,18000000,18000000,LF,60,1.000,228,77,18,56,0.338,1.067,ATL,18000000,35,60,0.583333
9,Travis d'Arnaud,16000000,8000000,C,44,0.733,165,53,9,34,0.321,0.919,ATL,8000000,35,60,0.583333
